In [2]:
import pandas as pd
import numpy as np
import math
import torch
from torch.nn import Module
import torch.nn.functional as F
from gpytorch.kernels import Kernel
from gpytorch.constraints import Positive
from gpytorch.lazy import MatmulLazyTensor, RootLazyTensor
from gpytorch.functions import RBFCovariance
from gpytorch.settings import trace_mode
from gpytorch.kernels.kernel import Kernel
import csv
import torch
import numpy as np
import json
import branca
import folium
import arrow
import gpytorch
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
from gpytorch.means.mean import Mean                                            
import random
#from DeepKernel import DeepNonstationarySpatiotemporalKernel, FocusPointsNN
from torch.nn.utils import clip_grad_norm_
%matplotlib inline
import random
import torch
from torch.nn import Module
import torch.nn.functional as F
from gpytorch.kernels import Kernel
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_auc_score, roc_curve
from scipy import interp
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import ADASYN
import ipywidgets as widgets
from IPython.display import display, clear_output
import geopandas as gpd
import ipywidgets as widgets
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

In [36]:
df = pd.read_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\Updated_mobility_data.csv", encoding='ISO-8859-1')
df.head(5)

,Meldedatum,bundesland,LandKreis_id,kreis,kr_ep_daily(inward_mobility),value,Percentage
0,01-01-2020,Schleswig-Holstein,1001,"Flensburg, Stadt",62.961749,6.0,5.801392
1,02-01-2020,Schleswig-Holstein,1001,"Flensburg, Stadt",62.961749,-7.7,5.801392
2,03-01-2020,Schleswig-Holstein,1001,"Flensburg, Stadt",62.961749,-9.7,5.801392
3,04-01-2020,Schleswig-Holstein,1001,"Flensburg, Stadt",62.961749,0.8,5.801392
4,05-01-2020,Schleswig-Holstein,1001,"Flensburg, Stadt",62.961749,4.6,5.801392


In [37]:

df['value_mobility_change'] = df['value'] * df['Percentage'] / 100

df.to_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\mobility_updated_15_11_2023.csv", index=False, encoding='ISO-8859-1')


#### updated_mobility_data.csv - is a merged dataset from pendler and taeglischemobilitaetesgeschen
#### mobility_updated_15_11_2023.csv - has new column "value_mobility_change". This dataset has berlin id 11000. This is not disaggregated yet.


x=pd.read_csv ("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\updated_mobility_data.csv", encoding='ISO-8859-1')
y=pd.read_csv ("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\mobility1_final.csv",encoding='ISO-8859-1')

In [38]:

x=pd.read_csv ("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\mobility_updated_15_11_2023.csv", encoding='ISO-8859-1')

#### z dataset is the final dataset for covid19. It has cases, deaths and Hotspot column

In [39]:
# combined 7-dayz incidenze dataset and Anzahltodesfall dataset (both from Zenodo RKI)
z=pd.read_csv ("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\hotspot_death1.csv")

In [40]:

if 'LandKreis_id' in x.columns and 'LandKreis_id' in z.columns:
    # Get the unique LandKreis_id values from both dataframes
    x_ids = set(x['LandKreis_id'].unique())
    z_ids = set(z['LandKreis_id'].unique())

    # Find LandKreis_id values that are in x but not in z
    missing_in_z = x_ids - z_ids

    # Find LandKreis_id values that are in z but not in x
    missing_in_x = z_ids - x_ids

    # Display the missing LandKreis_id values
    print("LandKreis_id missing in z:")
    print(missing_in_z)

    print("LandKreis_id missing in x:")
    print(missing_in_x)

else:
    print("One of the dataframes does not contain the 'LandKreis_id' column.")


LandKreis_id missing in z:
{11000}
LandKreis_id missing in x:
{11008, 11009, 11010, 11011, 11012, 11001, 11002, 11003, 11004, 11005, 11006, 11007}


#### so, the borough ids of Berlin like [11008, 11009, 11010, 11011, 11012, 11001, 11002, 11003, 11004, 11005, 11006, 11007} is missing in updated_Mobility_death1.csv.  But it has Berlin main id 11000 in it.


#### when Berlin value 11000 in mobility data is disaggregated to 11008, 11009, 11010, 11011, 11012, 11001, 11002, 11003, 11004, 11005, 11006, 11007. then we can merge it with covid data. so, all the data points for all landkreis ids are available.

In [41]:
mobility_data=pd.read_csv ("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\mobility_updated_15_11_2023.csv", encoding='ISO-8859-1')

#### Our Pendler data has mobility changes only for Berlin (as a whole) not for its individual Boroughs. So, I'm disaggregating the values to all 12 Boroughs of Berlin
                                          

In [42]:
berlin_data = mobility_data[
    (mobility_data['LandKreis_id'] == 11000) &
    (mobility_data['Meldedatum'] >= '01-01-2020') &
    (mobility_data['Meldedatum'] <= '31-12-2020')
]

# The list of new LandKreis_ids
new_landkreis_ids = [11008, 11009, 11010, 11011, 11012, 11001, 11002, 11003, 11004, 11005, 11006, 11007]

# Create a new dataframe to hold the disaggregated data
frames = []  # Use a list to collect dataframes

# Loop over the new LandKreis_ids
for lk_id in new_landkreis_ids:
    temp = berlin_data.copy()
    temp['LandKreis_id'] = lk_id  # Assign the new LandKreis_id
    
    # Assuming equal distribution among boroughs
    temp['value_mobility_change'] /= len(new_landkreis_ids)
    
    # Add temp to the list of frames
    frames.append(temp)

# Concatenate the new frames into one dataframe
disaggregated_data = pd.concat(frames, ignore_index=True)

# Filter out the original "11000" entries for 2020
mobility_data = mobility_data[
    ~((mobility_data['LandKreis_id'] == 11000) &
      (mobility_data['Meldedatum'] >= '01-01-2020') &
      (mobility_data['Meldedatum'] <= '31-12-2020'))
]
# Concatenate the original data with the disaggregated data
mobility_data = pd.concat([mobility_data, disaggregated_data], ignore_index=True)

# Save the result to a new CSV file
mobility_data.to_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\disaggregated_mobility_newdata.csv", index=False, encoding='ISO-8859-1')


In [3]:
df = pd.read_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\disaggregated_mobility_newdata.csv", encoding='ISO-8859-1')
df.head(5)

,Meldedatum,bundesland,LandKreis_id,kreis,kr_ep_daily(inward_mobility),value,Percentage,value_mobility_change
0,01-01-2020,Schleswig-Holstein,1001,"Flensburg, Stadt",62.961749,6.0,5.801392,0.348084
1,02-01-2020,Schleswig-Holstein,1001,"Flensburg, Stadt",62.961749,-7.7,5.801392,-0.446707
2,03-01-2020,Schleswig-Holstein,1001,"Flensburg, Stadt",62.961749,-9.7,5.801392,-0.562735
3,04-01-2020,Schleswig-Holstein,1001,"Flensburg, Stadt",62.961749,0.8,5.801392,0.046411
4,05-01-2020,Schleswig-Holstein,1001,"Flensburg, Stadt",62.961749,4.6,5.801392,0.266864


In [6]:
import pandas as pd

# Filter rows where 'value_mobility_change' is 0
rows_with_zero_mobility_change = df[df['value_mobility_change'] == 0]

# Display these rows
print(rows_with_zero_mobility_change)


        Meldedatum        bundesland  LandKreis_id                      kreis  \
5793    30-10-2020           Hamburg          2000                    Hamburg   
81451   18-07-2020  Freistaat Bayern          9161                 Ingolstadt   
81540   15-10-2020  Freistaat Bayern          9161                 Ingolstadt   
81817   18-07-2020  Freistaat Bayern          9184  München, Landeshauptstadt   
81906   15-10-2020  Freistaat Bayern          9184  München, Landeshauptstadt   
...            ...               ...           ...                        ...   
149211  14-09-2020            Berlin         11003                     Berlin   
149576  14-09-2020            Berlin         11004                     Berlin   
149941  14-09-2020            Berlin         11005                     Berlin   
150306  14-09-2020            Berlin         11006                     Berlin   
150671  14-09-2020            Berlin         11007                     Berlin   

        kr_ep_daily(inward_

In [4]:
df.isnull().sum()

Meldedatum                      0
bundesland                      0
LandKreis_id                    0
kreis                           0
kr_ep_daily(inward_mobility)    0
value                           0
Percentage                      0
value_mobility_change           0
dtype: int64

In [7]:
df.shape

(150780, 8)

#### This disaggregated_mobility_newdata.csv is the final mobility data after merging and processing. This data is now merged with hotspot_death1.csv to create neighbor_varaibles


In [47]:

merged_df = pd.merge(z,df, on=['LandKreis_id', 'Meldedatum'], how='outer')
merged_df.to_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\merged_data_Hotspot&Final_Mobility_data.csv", index=False)


In [20]:
df1 = pd.read_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\merged_data_Hotspot&Final_Mobility_data.csv")
df1.head(5)

,Meldedatum,LandKreis_id,Bevoelkerung,Faelle_gesamt,Faelle_neu,Faelle_7-Tage,Inzidenz_7-Tage,Hotspot,longitude,latitude,AnzahlTodesfall,bundesland,kreis,kr_ep_daily(inward_mobility),value,Percentage,value_mobility_change
0,03-01-2020,1001,91113.0,0.0,0.0,0.0,0.0,0.0,9.44572,54.788333,0.0,Schleswig-Holstein,"Flensburg, Stadt",62.961749,-9.7,5.801392,-0.562735
1,04-01-2020,1001,91113.0,0.0,0.0,0.0,0.0,0.0,9.44572,54.788333,0.0,Schleswig-Holstein,"Flensburg, Stadt",62.961749,0.8,5.801392,0.046411
2,05-01-2020,1001,91113.0,0.0,0.0,0.0,0.0,0.0,9.44572,54.788333,0.0,Schleswig-Holstein,"Flensburg, Stadt",62.961749,4.6,5.801392,0.266864
3,06-01-2020,1001,91113.0,0.0,0.0,0.0,0.0,0.0,9.44572,54.788333,0.0,Schleswig-Holstein,"Flensburg, Stadt",62.961749,-0.2,5.801392,-0.011603
4,07-01-2020,1001,91113.0,0.0,0.0,0.0,0.0,0.0,9.44572,54.788333,0.0,Schleswig-Holstein,"Flensburg, Stadt",62.961749,7.1,5.801392,0.411899


In [21]:
import pandas as pd

# Filter rows where 'value_mobility_change' is 0
rows_with_zero_mobility_change = df1[df1['value_mobility_change'] == 0]

# Display these rows
print(len(rows_with_zero_mobility_change))

284


In [22]:
df1.shape


(150792, 17)

In [23]:
df1.isnull().sum()

Meldedatum                        0
LandKreis_id                      0
Bevoelkerung                    824
Faelle_gesamt                   824
Faelle_neu                      824
Faelle_7-Tage                   824
Inzidenz_7-Tage                 824
Hotspot                         824
longitude                       824
latitude                        824
AnzahlTodesfall                 824
bundesland                       12
kreis                            12
kr_ep_daily(inward_mobility)     12
value                            12
Percentage                       12
value_mobility_change            12
dtype: int64

#### Mobility_data columns missing 12 values because the data point for the date 08-05-2020 in pendler data for Berlin is missing. so, the date belong to its boroughs are also missing

#### covid data columns has some missing values of 824 each , because of the missing data points for 01-01-2020 and 02-01-2020. These rows can be deleted because, the covid virus started to evolve only from March in germany.

### So, I'm dropping these rows.

In [24]:

original_shape = df1.shape
print(f'Original shape: {original_shape}')

# Drop rows with any null values
df1_cleaned = df1.dropna()

# Check the shape after dropping rows with null values
new_shape = df1_cleaned.shape
print(f'New shape after dropping rows with null values: {new_shape}')

# Save the cleaned dataframe back to CSV if needed
df1_cleaned.to_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\Cleaned_data_Hotspot&Final_Mobility_02.csv", index=False)



Original shape: (150792, 17)
New shape after dropping rows with null values: (149956, 17)


In [25]:
df4 = pd.read_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\Cleaned_data_Hotspot&Final_Mobility_02.csv")
df4.shape

(149956, 17)

In [26]:
import pandas as pd

# Filter rows where 'value_mobility_change' is 0
rows_with_zero_mobility_change = df4[df4['value_mobility_change'] == 0]

# Display these rows
print(len(rows_with_zero_mobility_change))

284


### so, finally covid with mobility dataset Finally_merged_data_Hotspot&Final_Mobility_data.csv is ready. now, this is used to create an neighbouring_variable to support spatial dynamics for my model.

In [29]:
import pandas as pd

# Read the dataset
df = pd.read_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\Cleaned_data_Hotspot&Final_Mobility_02.csv")

# Ensure the LandKreis_id is formatted with leading zeros
df['LandKreis_id'] = df['LandKreis_id'].apply(lambda x: f"{x:05d}")
berlin_boroughs = ['11001', '11002', '11003','11004', '11005','11006','11007','11008','11009','11010','11011','11012']
# Sample neighbors data
neighbors_data = {'01001': ['01059'], '01002': ['01057', '01058'], '01003': ['01053', '01055', '01062', '13074'], '01004': ['01057', '01058', '01060'], '01051': ['01054', '01058', '01059', '01061'], '01053': ['01003', '01062', '02000', '03353', '03355', '13074', '13076'], '01054': ['01051', '01059'], '01055': ['01003', '01057', '01060', '01062'], '01056': ['01060', '01061', '02000'], '01057': ['01002', '01004', '01055', '01058', '01060'], '01058': ['01002', '01004', '01051', '01057', '01059', '01060', '01061'], '01059': ['01001', '01051', '01054', '01058'], '01060': ['01004', '01055', '01056', '01057', '01058', '01061', '01062', '02000'], '01061': ['01051', '01056', '01058', '01060'], '01062': ['01003', '01053', '01055', '01060', '02000'], '02000': ['01053', '01056', '01060', '01062', '03353', '03359'], '03101': ['03102', '03151', '03154', '03157', '03158'], '03102': ['03101', '03153', '03157', '03158', '03254'], '03103': ['03151', '03154'], '03151': ['03101', '03103', '03154', '03157', '03241', '03351', '03360', '15081', '15083'], '03153': ['03102', '03155', '03158', '03159', '03254', '15085', '16062'], '03154': ['03101', '03103', '03151', '03158', '15083', '15085'], '03155': ['03153', '03159', '03254', '03255', '05762', '06633'], '03157': ['03101', '03102', '03151', '03241', '03254'], '03158': ['03101', '03102', '03153', '03154', '03254', '15085'], '03159': ['03153', '03155', '06611', '06633', '06636', '16061', '16062'], '03241': ['03151', '03157', '03252', '03254', '03256', '03257', '03351', '03358'], '03251': ['03256', '03361', '03401', '03458', '03459', '03460', '04011', '05770'], '03252': ['03241', '03254', '03255', '03257', '05766'], '03254': ['03102', '03153', '03155', '03157', '03158', '03241', '03252', '03255'], '03255': ['03155', '03252', '03254', '05762', '05766'], '03256': ['03241', '03251', '03257', '03358', '03361', '05770'], '03257': ['03241', '03252', '03256', '05766', '05770'], '03351': ['03151', '03241', '03358', '03360'], '03352': ['03356', '03357', '03359', '03461', '04012'], '03353': ['01053', '02000', '03355', '03357', '03358', '03359'], '03354': ['03355', '03360', '12070', '13076', '15081', '15090'], '03355': ['01053', '03353', '03354', '03358', '03360', '13076'], '03356': ['03352', '03357', '03361', '03461', '04011'], '03357': ['03352', '03353', '03356', '03358', '03359', '03361'], '03358': ['03241', '03256', '03351', '03353', '03355', '03357', '03360', '03361'], '03359': ['02000', '03352', '03353', '03357'], '03360': ['03151', '03351', '03354', '03355', '03358', '15081'], '03361': ['03251', '03256', '03356', '03357', '03358', '04011'], '03401': ['03251', '03458', '03461', '04011'], '03402': ['03452', '03457'], '03403': ['03451', '03458', '03461'], '03404': ['03459', '05566'], '03405': ['03455'], '03451': ['03403', '03453', '03455', '03457', '03458', '03461'], '03452': ['03402', '03457', '03462'], '03453': ['03451', '03454', '03457', '03458', '03459', '03460'], '03454': ['03453', '03456', '03457', '03459', '05566'], '03455': ['03405', '03451', '03457', '03461', '03462'], '03456': ['03454', '05554', '05566'], '03457': ['03402', '03451', '03452', '03453', '03454', '03455', '03462'], '03458': ['03251', '03401', '03403', '03451', '03453', '03460', '03461'], '03459': ['03251', '03404', '03453', '03454', '03460', '05566', '05570', '05754', '05758', '05770'], '03460': ['03251', '03453', '03458', '03459'], '03461': ['03352', '03356', '03401', '03403', '03451', '03455', '03458', '04011', '04012'], '03462': ['03452', '03455', '03457'], '04011': ['03251', '03356', '03361', '03401', '03461'], '04012': ['03352', '03461'], '05111': ['05112', '05158', '05162'], '05112': ['05111', '05114', '05117', '05119', '05158', '05162', '05170'], '05113': ['05117', '05119', '05158', '05512', '05513', '05562', '05911', '05954'], '05114': ['05112', '05162', '05166', '05170'], '05116': ['05162', '05166', '05370'], '05117': ['05112', '05113', '05119', '05158'], '05119': ['05112', '05113', '05117', '05170', '05512'], '05120': ['05122', '05124', '05374', '05378'], '05122': ['05120', '05124', '05158', '05378'], '05124': ['05120', '05122', '05158', '05374', '05954'], '05154': ['05166', '05170', '05554'], '05158': ['05111', '05112', '05113', '05117', '05122', '05124', '05162', '05315', '05316', '05378', '05954'], '05162': ['05111', '05112', '05114', '05116', '05158', '05166', '05315', '05358', '05362', '05370'], '05166': ['05114', '05116', '05154', '05162', '05170', '05370'], '05170': ['05112', '05114', '05119', '05154', '05166', '05512', '05554', '05562'], '05314': ['05382', '07131'], '05315': ['05158', '05162', '05316', '05362', '05378', '05382'], '05316': ['05158', '05315', '05378'], '05334': ['05358', '05366', '05370'], '05358': ['05162', '05334', '05362', '05366', '05370'], '05362': ['05162', '05315', '05358', '05366', '05382'], '05366': ['05334', '05358', '05362', '05382', '07131', '07232', '07233'], '05370': ['05116', '05162', '05166', '05334', '05358'], '05374': ['05120', '05124', '05378', '05382', '05954', '05962', '05966', '07132'], '05378': ['05120', '05122', '05158', '05315', '05316', '05374', '05382'], '05382': ['05314', '05315', '05362', '05366', '05374', '05378', '07131', '07132', '07138'], '05512': ['05113', '05119', '05170', '05562'], '05513': ['05113', '05562', '05911', '05916'], '05515': ['05558', '05566', '05570'], '05554': ['03456', '05154', '05170', '05558', '05562', '05566'], '05558': ['05515', '05554', '05562', '05566', '05570', '05915', '05978'], '05562': ['05113', '05170', '05512', '05513', '05554', '05558', '05911', '05913', '05916', '05978'], '05566': ['03404', '03454', '03456', '03459', '05515', '05554', '05558', '05570'], '05570': ['03459', '05515', '05558', '05566', '05754', '05915', '05974'], '05711': ['05754', '05758', '05766'], '05754': ['03459', '05570', '05711', '05758', '05766', '05774', '05974'], '05758': ['03459', '05711', '05754', '05766', '05770'], '05762': ['03155', '03255', '05766', '05774', '05958', '06633', '06635'], '05766': ['03252', '03255', '03257', '05711', '05754', '05758', '05762', '05770', '05774'], '05770': ['03251', '03256', '03257', '03459', '05758', '05766'], '05774': ['05754', '05762', '05766', '05958', '05974'], '05911': ['05113', '05513', '05562', '05913', '05916', '05954'], '05913': ['05562', '05911', '05914', '05954', '05978'], '05914': ['05913', '05954', '05962', '05978'], '05915': ['05558', '05570', '05974', '05978'], '05916': ['05513', '05562', '05911'], '05954': ['05113', '05124', '05158', '05374', '05911', '05913', '05914', '05962'], '05958': ['05762', '05774', '05962', '05966', '05970', '05974', '06635'], '05962': ['05374', '05914', '05954', '05958', '05966', '05974', '05978'], '05966': ['05374', '05958', '05962', '05970', '07132'], '05970': ['05958', '05966', '06532', '06534', '06635', '07132', '07143'], '05974': ['05570', '05754', '05774', '05915', '05958', '05962', '05978'], '05978': ['05558', '05562', '05913', '05914', '05915', '05962', '05974'], '06411': ['06432', '06438'], '06412': ['06413', '06433', '06434', '06435', '06436', '06438', '06440'], '06413': ['06412', '06435', '06438'], '06414': ['06433', '06436', '06439', '07315', '07339'], '06431': ['06432', '06433', '06437', '07311', '07319', '07331', '07338', '08222', '08226'], '06432': ['06411', '06431', '06433', '06437', '06438', '09671', '09676'], '06433': ['06412', '06414', '06431', '06432', '06436', '06438', '07315', '07319', '07331', '07339'], '06434': ['06412', '06436', '06439', '06440', '06532', '06533'], '06435': ['06412', '06413', '06438', '06440', '06535', '06631', '09671', '09672', '09677'], '06436': ['06412', '06414', '06433', '06434', '06439'], '06437': ['06431', '06432', '08225', '08226', '09676'], '06438': ['06411', '06412', '06413', '06432', '06433', '06435', '09671'], '06439': ['06414', '06434', '06436', '06533', '07141', '07339'], '06440': ['06412', '06434', '06435', '06531', '06532', '06535'], '06531': ['06440', '06532', '06534', '06535'], '06532': ['05970', '06434', '06440', '06531', '06533', '06534', '07143'], '06533': ['06434', '06439', '06532', '07141', '07143'], '06534': ['05970', '06531', '06532', '06535', '06634', '06635'], '06535': ['06435', '06440', '06531', '06534', '06631', '06632', '06634'], '06611': ['03159', '06633'], '06631': ['06435', '06535', '06632', '09672', '09673', '16063', '16066'], '06632': ['06535', '06631', '06634', '06636', '16063'], '06633': ['03155', '03159', '05762', '06611', '06634', '06635', '06636'], '06634': ['06534', '06535', '06632', '06633', '06635', '06636'], '06635': ['05762', '05958', '05970', '06534', '06633', '06634'], '06636': ['03159', '06632', '06633', '06634', '16061', '16063', '16064'], '07111': ['07137', '07141', '07143'], '07131': ['05314', '05366', '05382', '07137', '07138', '07233'], '07132': ['05374', '05382', '05966', '05970', '07138', '07143'], '07133': ['07134', '07140', '07331', '07333', '07336', '07339'], '07134': ['07133', '07140', '07231', '07235', '07336', '10046'], '07135': ['07137', '07140', '07231', '07233'], '07137': ['07111', '07131', '07135', '07138', '07140', '07141', '07143', '07233'], '07138': ['05382', '07131', '07132', '07137', '07143'], '07140': ['07133', '07134', '07135', '07137', '07141', '07231', '07339'], '07141': ['06439', '06533', '07111', '07137', '07140', '07143', '07339'], '07143': ['05970', '06532', '06533', '07111', '07132', '07137', '07138', '07141'], '07211': ['07235'], '07231': ['07134', '07135', '07140', '07232', '07233', '07235'], '07232': ['05366', '07231', '07233', '07235'], '07233': ['05366', '07131', '07135', '07137', '07231', '07232'], '07235': ['07134', '07211', '07231', '07232', '10042', '10046'], '07311': ['06431', '07314', '07338', '08222'], '07312': ['07332', '07335'], '07313': ['07332', '07337', '07340'], '07314': ['07311', '07338', '08222'], '07315': ['06414', '06433', '07339'], '07316': ['07332', '07337', '07338'], '07317': ['07340'], '07318': ['07338', '08215', '08226'], '07319': ['06431', '06433', '07331', '07332', '07338'], '07320': ['07340', '10045'], '07331': ['06431', '06433', '07133', '07319', '07332', '07333', '07339'], '07332': ['07312', '07313', '07316', '07319', '07331', '07333', '07335', '07337', '07338', '07340'], '07333': ['07133', '07331', '07332', '07335', '07336'], '07334': ['07337', '07338', '08212', '08215', '08216'], '07335': ['07312', '07332', '07333', '07336', '07340', '10045'], '07336': ['07133', '07134', '07333', '07335', '07340', '10043', '10045', '10046'], '07337': ['07313', '07316', '07332', '07334', '07338', '07340'], '07338': ['06431', '07311', '07314', '07316', '07318', '07319', '07332', '07334', '07337', '08215', '08222', '08226'], '07339': ['06414', '06433', '06439', '07133', '07140', '07141', '07315', '07331'], '07340': ['07313', '07317', '07320', '07332', '07335', '07336', '07337', '10045'], '08111': ['08115', '08116', '08118', '08119'], '08115': ['08111', '08116', '08118', '08235', '08236', '08415', '08416'], '08116': ['08111', '08115', '08117', '08119', '08415', '08425'], '08117': ['08116', '08119', '08135', '08136', '08415', '08425'], '08118': ['08111', '08115', '08119', '08125', '08236'], '08119': ['08111', '08116', '08117', '08118', '08125', '08127', '08136'], '08121': ['08125'], '08125': ['08118', '08119', '08121', '08126', '08127', '08215', '08225', '08226', '08236'], '08126': ['08125', '08127', '08128', '08225'], '08127': ['08119', '08125', '08126', '08128', '08136', '09571'], '08128': ['08126', '08127', '08225', '09571', '09575', '09676', '09677', '09679'], '08135': ['08117', '08136', '08425', '09773', '09774', '09779'], '08136': ['08117', '08119', '08127', '08135', '09571', '09779'], '08211': ['08216'], '08212': ['07334', '08215'], '08215': ['07318', '07334', '07338', '08125', '08212', '08216', '08226', '08235', '08236'], '08216': ['07334', '08211', '08215', '08235', '08237', '08317'], '08221': ['08222', '08226'], '08222': ['06431', '07311', '07314', '07338', '08221', '08226'], '08225': ['06437', '08125', '08126', '08128', '08226', '09676'], '08226': ['06431', '06437', '07318', '07338', '08125', '08215', '08221', '08222', '08225'], '08231': ['08235', '08236'], '08235': ['08115', '08215', '08216', '08231', '08236', '08237', '08416'], '08236': ['08115', '08118', '08125', '08215', '08231', '08235'], '08237': ['08216', '08235', '08317', '08325', '08416', '08417'], '08311': ['08315', '08316'], '08315': ['08311', '08316', '08326', '08336', '08337'], '08316': ['08311', '08315', '08317', '08326'], '08317': ['08216', '08237', '08316', '08325', '08326'], '08325': ['08237', '08317', '08326', '08327', '08417'], '08326': ['08315', '08316', '08317', '08325', '08327', '08335', '08337'], '08327': ['08325', '08326', '08335', '08417', '08437'], '08335': ['08326', '08327', '08435', '08437'], '08336': ['08315', '08337'], '08337': ['08315', '08326', '08336'], '08415': ['08115', '08116', '08117', '08416', '08417', '08425', '08426', '08437'], '08416': ['08115', '08235', '08237', '08415', '08417'], '08417': ['08237', '08325', '08327', '08415', '08416', '08437'], '08421': ['08425', '09775'], '08425': ['08116', '08117', '08135', '08415', '08421', '08426', '09774', '09775'], '08426': ['08415', '08425', '08436', '08437', '09764', '09775', '09778'], '08435': ['08335', '08436', '08437', '09776'], '08436': ['08426', '08435', '08437', '09764', '09776', '09778', '09780'], '08437': ['08327', '08335', '08415', '08417', '08426', '08435', '08436'], '09161': ['09176', '09185', '09186'], '09162': ['09174', '09179', '09184'], '09163': ['09187'], '09171': ['09183', '09189', '09277'], '09172': ['09189'], '09173': ['09180', '09182', '09184', '09188', '09190'], '09174': ['09162', '09178', '09179', '09184', '09186', '09771'], '09175': ['09177', '09183', '09184', '09187'], '09176': ['09161', '09185', '09186', '09273', '09373', '09576', '09577', '09779'], '09177': ['09175', '09178', '09183', '09184', '09274'], '09178': ['09174', '09177', '09184', '09186', '09273', '09274'], '09179': ['09162', '09174', '09181', '09184', '09188', '09771'], '09180': ['09173', '09190', '09777'], '09181': ['09179', '09188', '09190', '09771', '09772', '09777'], '09182': ['09173', '09184', '09187'], '09183': ['09171', '09175', '09177', '09187', '09189', '09274', '09277'], '09184': ['09162', '09173', '09174', '09175', '09177', '09178', '09179', '09182', '09187', '09188'], '09185': ['09161', '09176', '09186', '09771', '09779'], '09186': ['09161', '09174', '09176', '09178', '09185', '09273', '09771'], '09187': ['09163', '09175', '09182', '09183', '09184', '09189'], '09188': ['09173', '09179', '09181', '09184', '09190'], '09189': ['09171', '09172', '09183', '09187'], '09190': ['09173', '09180', '09181', '09188', '09777'], '09261': ['09274'], '09262': ['09275'], '09263': ['09278'], '09271': ['09272', '09275', '09276', '09277', '09278', '09279'], '09272': ['09271', '09275', '09276'], '09273': ['09176', '09178', '09186', '09274', '09373', '09375'], '09274': ['09177', '09178', '09183', '09261', '09273', '09277', '09278', '09279', '09375'], '09275': ['09262', '09271', '09272', '09277'], '09276': ['09271', '09272', '09278', '09372'], '09277': ['09171', '09183', '09271', '09274', '09275', '09279'], '09278': ['09263', '09271', '09274', '09276', '09279', '09372', '09375'], '09279': ['09271', '09274', '09277', '09278'], '09361': ['09371'], '09362': ['09375'], '09363': ['09374'], '09371': ['09361', '09373', '09374', '09376', '09472', '09574'], '09372': ['09276', '09278', '09375', '09376'], '09373': ['09176', '09273', '09371', '09375', '09376', '09574', '09576'], '09374': ['09363', '09371', '09376', '09377', '09472'], '09375': ['09273', '09274', '09278', '09362', '09372', '09373', '09376'], '09376': ['09371', '09372', '09373', '09374', '09375'], '09377': ['09374', '09472', '09479'], '09461': ['09471'], '09462': ['09472'], '09463': ['09473'], '09464': ['09475'], '09471': ['09461', '09472', '09473', '09474', '09478', '09572', '09575', '09674', '09675', '09678'], '09472': ['09371', '09374', '09377', '09462', '09471', '09474', '09475', '09477', '09478', '09479', '09574'], '09473': ['09463', '09471', '09476', '09478', '09674', '16069', '16072'], '09474': ['09471', '09472', '09572', '09574'], '09475': ['09464', '09472', '09476', '09477', '09479', '14523', '16075'], '09476': ['09473', '09475', '09477', '09478', '16072', '16073', '16075'], '09477': ['09472', '09475', '09476', '09478'], '09478': ['09471', '09472', '09473', '09476', '09477'], '09479': ['09377', '09472', '09475'], '09561': ['09571'], '09562': ['09563', '09564', '09572', '09573'], '09563': ['09562', '09564', '09572', '09573'], '09564': ['09562', '09563', '09565', '09572', '09573', '09574', '09576'], '09565': ['09564', '09573', '09576'], '09571': ['08127', '08128', '08136', '09561', '09573', '09575', '09576', '09577', '09779'], '09572': ['09471', '09474', '09562', '09563', '09564', '09573', '09574', '09575'], '09573': ['09562', '09563', '09564', '09565', '09571', '09572', '09575', '09576'], '09574': ['09371', '09373', '09472', '09474', '09564', '09572', '09576'], '09575': ['08128', '09471', '09571', '09572', '09573', '09675', '09679'], '09576': ['09176', '09373', '09564', '09565', '09571', '09573', '09574', '09577'], '09577': ['09176', '09571', '09576', '09779'], '09661': ['09671', '09676'], '09662': ['09678'], '09663': ['09679'], '09671': ['06432', '06435', '06438', '09661', '09676', '09677'], '09672': ['06435', '06631', '09673', '09677', '09678'], '09673': ['06631', '09672', '09674', '09678', '16066', '16069'], '09674': ['09471', '09473', '09673', '09678', '16069'],
                  '09675': ['09471', '09575', '09678', '09679'], '09676': ['06432', '06437', '08128', '08225', '09661', '09671', '09677'], '09677': ['06435', '08128', '09671', '09672', '09676', '09678', '09679'], '09678': ['09471', '09662', '09672', '09673', '09674', '09675', '09677', '09679'], '09679': ['08128', '09575', '09663', '09675', '09677', '09678'], '09761': ['09771', '09772'], '09762': ['09777'], '09763': ['09780'], '09764': ['08426', '08436', '09778'], '09771': ['09174', '09179', '09181', '09185', '09186', '09761', '09772', '09779'], '09772': ['09181', '09761', '09771', '09773', '09774', '09777', '09778', '09779'], 
                  '09773': ['08135', '09772', '09774', '09779'], '09774': ['08135', '08425', '09772', '09773', '09775', '09778'], '09775': ['08421', '08425', '08426', '09774', '09778'], '09776': ['08435', '08436', '09780'], '09777': ['09180', '09181', '09190', '09762', '09772', '09778', '09780'], '09778': ['08426', '08436', '09764', '09772', '09774', '09775', '09777', '09780'], '09779': ['08135', '08136', '09176', '09185', '09571', '09577', '09771', '09772', '09773'], '09780': ['08436', '09763', '09776', '09777', '09778'], '10041': ['10043', '10044', '10045'], '10042': ['07235', '10044', '10046'], '10043': ['07336', '10041', '10044', '10045', '10046'], '10044': ['10041', '10042', '10043', '10046'], '10045': ['07320', '07335', '07336', '07340', '10041', '10043'], '10046': ['07134', '07235', '07336', '10042', '10043', '10044'], '11000': ['12054', '12060', '12061', '12063', '12064', '12065', '12067', '12069', '12072'], '12051': ['12063', '12069'], '12052': ['12071'], '12053': ['12064', '12067'], '12054': ['11000', '12063', '12069'], '12060': ['11000', '12064', '12065', '12073'], '12061': ['11000', '12062', '12066', '12067', '12071', '12072'], '12062': ['12061', '12066', '12072', '14627', '14730', '15091'], '12063': ['11000', '12051', '12054', '12065', '12068', '12069', '15086', '15090'], '12064': ['11000', '12053', '12060', '12067'], '12065': ['11000', '12060', '12063', '12068', '12073', '13071'], '12066': ['12061', '12062', '12071', '14625', '14627'], '12067': ['11000', '12053', '12061', '12064', '12071'], '12068': ['12063', '12065', '12070', '13071', '13076', '15090'], '12069': ['11000', '12051', '12054', '12063', '12072', '15082', '15086', '15091'], '12070': ['03354', '12068', '13071', '13076', '15090'], '12071': ['12052', '12061', '12066', '12067', '14625', '14626'], '12072': ['11000', '12061', '12062', '12069', '15091'], '12073': ['12060', '12065', '13071', '13075'], '13003': ['13072'], '13004': ['13074', '13076'], '13071': ['12065', '12068', '12070', '12073', '13072', '13073', '13075', '13076'], '13072': ['13003', '13071', '13073', '13074', '13076'], '13073': ['13071', '13072', '13075'], '13074': ['01003', '01053', '13004', '13072', '13076'], '13075': ['12073', '13071', '13073'], '13076': ['01053', '03354', '03355', '12068', '12070', '13004', '13071', '13072', '13074'], '14511': ['14521', '14522', '14524'], 
                  '14521': ['14511', '14522', '14523', '14524'], '14522': ['14511', '14521', '14524', '14627', '14628', '14729', '14730', '16077'], '14523': ['09475', '14521', '14524', '16075', '16076'], '14524': ['14511', '14521', '14522', '14523', '16076', '16077'], '14612': ['14625', '14627', '14628'], '14625': ['12066', '12071', '14612', '14626', '14627', '14628'], '14626': ['12071', '14625'], '14627': ['12062', '12066', '14522', '14612', '14625', '14628', '14730'], '14628': ['14522', '14612', '14625', '14627'], '14713': ['14729', '14730'], '14729': ['14522', '14713', '14730', '15084', '15088', '16077'], '14730': ['12062', '14522', '14627', '14713', '14729', '15082', '15088', '15091'], '15001': ['15082', '15091'], '15002': ['15088'], '15003': ['15083', '15086', '15089'], '15081': ['03151', '03354', '03360', '15083', '15090'], '15082': ['12069', '14730', '15001', '15086', '15088', '15089', '15091'], '15083': ['03151', '03154', '15003', '15081', '15085', '15086', '15089', '15090'], '15084': ['14729', '15088', '16052', '16065', '16068', '16071', '16074', '16076', '16077'], '15085': ['03153', '03154', '03158', '15083', '15087', '15089', '16062'], '15086': ['12063', '12069', '15003', '15082', '15083', '15089', '15090'], '15087': ['15085', '15088', '15089', '16062', '16065'], '15088': ['14729', '14730', '15002', '15082', '15084', '15087', '15089', '16065'], '15089': ['15003', '15082', '15083', '15085', '15086', '15087', '15088'], '15090': ['03354', '12063', '12068', '12070', '15081', '15083', '15086'], '15091': ['12062', '12069', '12072', '14730', '15001', '15082'], '16051': ['16067', '16068', '16070', '16071'], '16052': ['15084', '16074', '16076'], '16053': ['16071', '16074'], '16054': ['16066', '16069', '16070'], '16055': ['16071'], '16061': ['03159', '06636', '16062', '16064', '16065'], '16062': ['03153', '03159', '15085', '15087', '16061', '16065'], '16063': ['06631', '06632', '06636', '16064', '16066', '16067'], '16064': ['06636', '16061', '16063', '16065', '16067', '16068'], '16065': ['15084', '15087', '15088', '16061', '16062', '16064', '16068'], '16066': ['06631', '09673', '16054', '16063', '16067', '16069', '16070'], '16067': ['16051', '16063', '16064', '16066', '16068', '16070'], '16068': ['15084', '16051', '16064', '16065', '16067', '16071'], 
                  '16069': ['09473', '09673', '09674', '16054', '16066', '16070', '16072', '16073'], '16070': ['16051', '16054', '16066', '16067', '16069', '16071', '16073'], '16071': ['15084', '16051', '16053', '16055', '16068', '16070', '16073', '16074'], '16072': ['09473', '09476', '16069', '16073'], '16073': ['09476', '16069', '16070', '16071', '16072', '16074', '16075'], '16074': ['15084', '16052', '16053', '16071', '16073', '16075', '16076'], '16075': ['09475', '09476', '14523', '16073', '16074', '16076'], '16076': ['14523', '14524', '15084', '16052', '16074', '16075', '16077'], '16077': ['14522', '14524', '14729', '15084', '16076'],
                  '11001': berlin_boroughs, '11002': berlin_boroughs,'11003': berlin_boroughs, '11004': berlin_boroughs,'11005': berlin_boroughs, '11006': berlin_boroughs,'11007': berlin_boroughs, '11008': berlin_boroughs,'11009': berlin_boroughs, '11010': berlin_boroughs,'11011': berlin_boroughs, '11012': berlin_boroughs
                 }

# Add new columns for aggregated data and initialize them as float
agg_columns = ['Faelle_gesamt', 'Faelle_neu', 'Faelle_7-Tage', 'Inzidenz_7-Tage', 'AnzahlTodesfall']
for col in agg_columns:
    df['Neighbor_' + col] = 0.0  # Initialize with float

for date in df['Meldedatum'].unique():
    date_df = df[df['Meldedatum'] == date]
    for index, row in date_df.iterrows():
        neighbors = neighbors_data.get(row['LandKreis_id'], [])
        for neighbor in neighbors:
            neighbor_data = date_df[date_df['LandKreis_id'] == neighbor]
            if not neighbor_data.empty:
                for col in agg_columns:
                    sum_value = neighbor_data[col].sum()  # Sum might be float
                    df.at[index, 'Neighbor_' + col] += float(sum_value)  # Explicit conversion to float

df.to_csv('C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\hotspot_neighbor_new.csv', index=False)


In [129]:
data1 = pd.read_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\hotspot_neighbor_new.csv")
data1.head(5)

,Meldedatum,LandKreis_id,Bevoelkerung,Faelle_gesamt,Faelle_neu,Faelle_7-Tage,Inzidenz_7-Tage,Hotspot,longitude,latitude,...,kreis,kr_ep_daily(inward_mobility),value,Percentage,value_mobility_change,Neighbor_Faelle_gesamt,Neighbor_Faelle_neu,Neighbor_Faelle_7-Tage,Neighbor_Inzidenz_7-Tage,Neighbor_AnzahlTodesfall
0,03-01-2020,1001,91113.0,0.0,0.0,0.0,0.0,0.0,9.44572,54.788333,...,"Flensburg, Stadt",62.961749,-9.7,5.801392,-0.562735,0.0,0.0,0.0,0.0,0.0
1,04-01-2020,1001,91113.0,0.0,0.0,0.0,0.0,0.0,9.44572,54.788333,...,"Flensburg, Stadt",62.961749,0.8,5.801392,0.046411,0.0,0.0,0.0,0.0,0.0
2,05-01-2020,1001,91113.0,0.0,0.0,0.0,0.0,0.0,9.44572,54.788333,...,"Flensburg, Stadt",62.961749,4.6,5.801392,0.266864,0.0,0.0,0.0,0.0,0.0
3,06-01-2020,1001,91113.0,0.0,0.0,0.0,0.0,0.0,9.44572,54.788333,...,"Flensburg, Stadt",62.961749,-0.2,5.801392,-0.011603,0.0,0.0,0.0,0.0,0.0
4,07-01-2020,1001,91113.0,0.0,0.0,0.0,0.0,0.0,9.44572,54.788333,...,"Flensburg, Stadt",62.961749,7.1,5.801392,0.411899,0.0,0.0,0.0,0.0,0.0


In [130]:
import pandas as pd

# Filter rows where 'value_mobility_change' is 0
rows_with_zero_mobility_change = data1[data1['value_mobility_change'] == 0]

# Display these rows
print(len(rows_with_zero_mobility_change))

284


#### hotspot_neighbor_new.csv is the very final dataset used in the model

In [76]:
data1.isnull().sum()

Meldedatum                      0
LandKreis_id                    0
Bevoelkerung                    0
Faelle_gesamt                   0
Faelle_neu                      0
Faelle_7-Tage                   0
Inzidenz_7-Tage                 0
Hotspot                         0
longitude                       0
latitude                        0
AnzahlTodesfall                 0
bundesland                      0
kreis                           0
kr_ep_daily(inward_mobility)    0
value                           0
Percentage                      0
value_mobility_change           0
Neighbor_Faelle_gesamt          0
Neighbor_Faelle_neu             0
Neighbor_Faelle_7-Tage          0
Neighbor_Inzidenz_7-Tage        0
Neighbor_AnzahlTodesfall        0
dtype: int64

In [77]:
data1.shape

(149956, 22)

In [78]:
data1.dtypes

Meldedatum                       object
LandKreis_id                      int64
Bevoelkerung                    float64
Faelle_gesamt                   float64
Faelle_neu                      float64
Faelle_7-Tage                   float64
Inzidenz_7-Tage                 float64
Hotspot                         float64
longitude                       float64
latitude                        float64
AnzahlTodesfall                 float64
bundesland                       object
kreis                            object
kr_ep_daily(inward_mobility)    float64
value                           float64
Percentage                      float64
value_mobility_change           float64
Neighbor_Faelle_gesamt          float64
Neighbor_Faelle_neu             float64
Neighbor_Faelle_7-Tage          float64
Neighbor_Inzidenz_7-Tage        float64
Neighbor_AnzahlTodesfall        float64
dtype: object

In [131]:
data1 = data1.drop(['Bevoelkerung','Faelle_gesamt','Faelle_7-Tage','bundesland', 'kreis','kr_ep_daily(inward_mobility)', 'value', 'Percentage', 'Neighbor_Faelle_gesamt','Neighbor_Faelle_7-Tage'], axis=1)

In [132]:
data1.shape

(149956, 12)

In [133]:
data1['7_days_lagged_hotspot'] = data1.groupby('LandKreis_id')['Hotspot'].shift(7)
data1.head(5)

,Meldedatum,LandKreis_id,Faelle_neu,Inzidenz_7-Tage,Hotspot,longitude,latitude,AnzahlTodesfall,value_mobility_change,Neighbor_Faelle_neu,Neighbor_Inzidenz_7-Tage,Neighbor_AnzahlTodesfall,7_days_lagged_hotspot
0,03-01-2020,1001,0.0,0.0,0.0,9.44572,54.788333,0.0,-0.562735,0.0,0.0,0.0,NaN
1,04-01-2020,1001,0.0,0.0,0.0,9.44572,54.788333,0.0,0.046411,0.0,0.0,0.0,NaN
2,05-01-2020,1001,0.0,0.0,0.0,9.44572,54.788333,0.0,0.266864,0.0,0.0,0.0,NaN
3,06-01-2020,1001,0.0,0.0,0.0,9.44572,54.788333,0.0,-0.011603,0.0,0.0,0.0,NaN
4,07-01-2020,1001,0.0,0.0,0.0,9.44572,54.788333,0.0,0.411899,0.0,0.0,0.0,NaN


In [134]:
class_counts = data1['7_days_lagged_hotspot'].value_counts()
class_counts

7_days_lagged_hotspot
0.0    124060
1.0     23019
Name: count, dtype: int64

In [135]:
data1.isnull().sum()

Meldedatum                     0
LandKreis_id                   0
Faelle_neu                     0
Inzidenz_7-Tage                0
Hotspot                        0
longitude                      0
latitude                       0
AnzahlTodesfall                0
value_mobility_change          0
Neighbor_Faelle_neu            0
Neighbor_Inzidenz_7-Tage       0
Neighbor_AnzahlTodesfall       0
7_days_lagged_hotspot       2877
dtype: int64

In [136]:
data1.shape

(149956, 13)

In [137]:
# Filter rows where 'value_mobility_change' is 0
rows_with_zero_mobility_change = data1[data1['value_mobility_change'] == 0]

# Display these rows
print(len(rows_with_zero_mobility_change))

284


##### now, imputed with knn imputer

In [138]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=7)  # you can change the number of neighbors

column_to_impute = data1[['7_days_lagged_hotspot']]

imputed_data = imputer.fit_transform(column_to_impute)

imputed_data_frame = pd.DataFrame(imputed_data, columns=['7_days_lagged_hotspot'])

# Replace the original column with the imputed column
data1['7_days_lagged_hotspot'] = imputed_data_frame['7_days_lagged_hotspot']
import numpy as np

data1['7_days_lagged_hotspot'] = np.round(imputed_data_frame['7_days_lagged_hotspot'])



In [139]:
data1.isnull().sum()

Meldedatum                  0
LandKreis_id                0
Faelle_neu                  0
Inzidenz_7-Tage             0
Hotspot                     0
longitude                   0
latitude                    0
AnzahlTodesfall             0
value_mobility_change       0
Neighbor_Faelle_neu         0
Neighbor_Inzidenz_7-Tage    0
Neighbor_AnzahlTodesfall    0
7_days_lagged_hotspot       0
dtype: int64

In [140]:
class_counts = data1['7_days_lagged_hotspot'].value_counts()
class_counts

7_days_lagged_hotspot
0.0    126937
1.0     23019
Name: count, dtype: int64

#### KNN imputer imputed with non-integer values. because it take mean of their neighbours [since, n_neighbours =5] but we need only 0s and 1s. !!!

#### when I rounded the imputed data from knn again there is no change in 1s and 0s count.

#### So, imputing with knn. with reason. mostly filled by 0s

In [141]:
data1.to_csv('C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\Data_final_19_12_2023.csv', index=False)

In [142]:
dat= pd.read_csv("C:\\Users\\Suji ramesh\\Desktop\\Thesis\\data\\Data_final_19_12_2023.csv")
dat.head(5)

,Meldedatum,LandKreis_id,Faelle_neu,Inzidenz_7-Tage,Hotspot,longitude,latitude,AnzahlTodesfall,value_mobility_change,Neighbor_Faelle_neu,Neighbor_Inzidenz_7-Tage,Neighbor_AnzahlTodesfall,7_days_lagged_hotspot
0,03-01-2020,1001,0.0,0.0,0.0,9.44572,54.788333,0.0,-0.562735,0.0,0.0,0.0,0.0
1,04-01-2020,1001,0.0,0.0,0.0,9.44572,54.788333,0.0,0.046411,0.0,0.0,0.0,0.0
2,05-01-2020,1001,0.0,0.0,0.0,9.44572,54.788333,0.0,0.266864,0.0,0.0,0.0,0.0
3,06-01-2020,1001,0.0,0.0,0.0,9.44572,54.788333,0.0,-0.011603,0.0,0.0,0.0,0.0
4,07-01-2020,1001,0.0,0.0,0.0,9.44572,54.788333,0.0,0.411899,0.0,0.0,0.0,0.0


In [143]:
dat.shape


(149956, 13)

In [144]:
# Filter rows where 'value_mobility_change' is 0
rows_with_zero_mobility_change = dat[dat['value_mobility_change'] == 0]

print(len(rows_with_zero_mobility_change))

284
